# Explore quants matrix   
3.31.22      

How can I design a simulated matrix that more closely resembles an actual peptides/protein quants matrix?    

In [1]:
import unittest
import pytest
import pandas as pd 
import numpy as np
import torch
from scipy.stats import ranksums

import sys
sys.path.append('../ms_imputer/')

from models.linear import GradNMFImputer
import util_functions

/Users/lincoln/miniconda3/envs/ms_imputer/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
quants_matrix = pd.read_csv("~/Desktop/maxquant-data/PXD010612_peptides.csv")
quants_matrix.replace([0, 0.0], np.nan, inplace=True)
quants_matrix = np.array(quants_matrix)
quants_matrix

array([[      nan,       nan,       nan, ...,  7348900.,  6215900.,
         7717400.],
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       [29276000., 24843000., 26687000., ..., 17439000., 17674000.,
        16140000.],
       ...,
       [      nan,       nan,       nan, ...,  2733700.,  2958100.,
         3523200.],
       [      nan,       nan,       nan, ..., 34843000., 34033000.,
        33496000.],
       [      nan,       nan,       nan, ..., 48650000., 22799000.,
        53835000.]])

In [3]:
print("matrix min: ", np.nanmin(quants_matrix))
print("matrix max: ", np.nanmax(quants_matrix))
print(" ")
print("matrix mean", np.nanmean(quants_matrix))
print("matrix std: ", np.nanstd(quants_matrix))

matrix min:  12096.0
matrix max:  61849000000.0
 
matrix mean 102161962.48006964
matrix std:  978349975.5876377


#### Init simulated matrix
Note that this matrix is likely no longer the designate rank, as we've taken the absolute value of gaussian drawn floats

In [4]:
def simulate_matrix_realistic(matrix_shape):
    """
    Init a simulated matrix of known size and (approximate) rank. 
    The values of quants_mean and quants_std were derived from a 
    real peptide quants matrix, and should allow us to generate a 
    matrix that more accurately simulates a real peptide quants 
    dataset. Note that taking the abs value of W and H most likely
    changes the true rank of the matrix, thus the assert statement
    in here won't necessarily pass. 
    
    Parameters
    ----------
    matrix_shape: tuple, (x,y,z) where x=n_rows, y=n_cols
                    and z=rank
    Returns
    -------
    X : np.ndarray, the simulated matrix
    """
    quants_mean = 102161962.5
    quants_std = 978349975.6

    matrix_shape = (12, 10, 3) # (n_rows, n_cols, rank)
    W = np.abs(np.random.normal(loc=quants_mean, scale=quants_std, size=(matrix_shape[0], matrix_shape[2])))
    H = np.abs(np.random.normal(loc=quants_mean, scale=quants_std, size=(matrix_shape[2], matrix_shape[1])))

    X = W @ H

    # won't necessarily pass
    assert np.linalg.matrix_rank(X) == matrix_shape[2]
    
    return X

In [5]:
# set params
matrix_shape = (200, 10, 3)
lf = 8
tolerance = 0.0001
batch_size = 100
max_iters = 3000
learning_rate = 0.1

matrix = simulate_matrix_realistic(matrix_shape)

train, val, test = util_functions.split(
                                    matrix,
                                    val_frac=0.1, 
                                    test_frac=0.1, 
                                    min_present=2
)

In [6]:
model = GradNMFImputer(
                n_rows=train.shape[0], 
                n_cols=train.shape[1], 
                n_factors=lf, 
                stopping_tol=tolerance, 
                train_batch_size=batch_size, 
                eval_batch_size=batch_size,
                n_epochs=max_iters, 
                loss_func="MSE",
                optimizer=torch.optim.Adam,
                optimizer_kwargs={"lr": learning_rate}
)

recon = model.fit_transform(train, val)

  4%|▍         | 122/3000 [00:00<00:05, 532.43epoch/s]

early stopping triggered


In [7]:
# rescale by a constant, for easier error tolerance calculation
train_scaled = train / 1e18
val_scaled = val / 1e18
test_scaled = test / 1e18
recon_scaled = recon / 1e18

train_err_scaled = util_functions.mse_func_np(train_scaled, recon_scaled)
val_err_scaled = util_functions.mse_func_np(val_scaled, recon_scaled)
test_err_scaled = util_functions.mse_func_np(test_scaled, recon_scaled)

print(train_err_scaled)
print(val_err_scaled)
print(test_err_scaled)

5.101520672503942e-15
0.29269762862028165
0.14894518651351432


In [8]:
print(train_err_scaled < 1e-6)
print(val_err_scaled < 1.0)
print(test_err_scaled < 1.0)

True
True
True


***

### Want to add np.nans to the matrix

In [44]:
n_nans = int(np.floor(matrix.size * 0.4))

In [45]:
# flatten the initial matrix
m_flat = matrix.flatten()
# randomly select indices
rand_idx = np.random.choice(len(m_flat), size=n_nans, replace=False)
# set to np.nans
m_flat[rand_idx] = np.nan
# reshape
matrix_missing = m_flat.reshape(matrix.shape)
matrix_missing

array([[9.60604445e+17, 8.36029026e+17, 8.70780404e+17, 7.75666024e+17,
                   nan, 6.69681962e+17, 4.79511110e+17,            nan,
                   nan, 1.86781233e+18],
       [6.32887491e+17,            nan, 1.60494453e+18,            nan,
                   nan,            nan,            nan, 2.06437852e+18,
        3.34708124e+18, 2.35055954e+18],
       [5.60389777e+17, 8.29221103e+17,            nan,            nan,
                   nan, 6.44143383e+17, 6.50912452e+17, 1.59377534e+18,
        1.97275247e+18, 1.72646066e+18],
       [1.26200757e+18,            nan, 4.75053748e+17,            nan,
        3.99552169e+17, 1.00027105e+18, 8.15807338e+17, 2.31472404e+18,
                   nan,            nan],
       [1.90653427e+18, 1.80065150e+18, 1.79689460e+18, 1.61492270e+18,
                   nan, 1.45554573e+18,            nan, 3.60692538e+18,
        3.06216271e+18,            nan],
       [           nan,            nan, 2.43497784e+18, 1.13119935e+18,
   

In [46]:
train, val, test = util_functions.split(
                                    matrix_missing,
                                    val_frac=0.1, 
                                    test_frac=0.1, 
                                    min_present=1
)

model = GradNMFImputer(
                n_rows=train.shape[0], 
                n_cols=train.shape[1], 
                n_factors=lf, 
                stopping_tol=tolerance, 
                train_batch_size=batch_size, 
                eval_batch_size=batch_size,
                n_epochs=max_iters, 
                loss_func="MSE",
                optimizer=torch.optim.Adam,
                optimizer_kwargs={"lr": learning_rate}
)

recon = model.fit_transform(train, val)

  2%|▏         | 66/3000 [00:00<00:05, 571.65epoch/s]

early stopping triggered


In [47]:
# rescale by a constant, for easier error tolerance calculation
train_scaled = train / 1e18
val_scaled = val / 1e18
test_scaled = test / 1e18
recon_scaled = recon / 1e18

train_err_scaled = util_functions.mse_func_np(train_scaled, recon_scaled)
val_err_scaled = util_functions.mse_func_np(val_scaled, recon_scaled)
test_err_scaled = util_functions.mse_func_np(test_scaled, recon_scaled)

print(train_err_scaled)
print(val_err_scaled)
print(test_err_scaled)

8.099204922858711e-15
0.13173816069595873
1.0310213437878208
